In [1]:
include("../src/ProcessSequence.jl")

Main.ProcessSequence

In [2]:
project_dir = dirname(@__DIR__)

using JSON: parse
project_json = parse(read(joinpath(project_dir, "project.json"), String))

sample_name = project_json["sample_name"]

output_dir = joinpath(project_dir, "output")
input_dir = joinpath(project_dir, "input")
sample_dir = joinpath(input_dir, sample_name)
reference_dir = joinpath(input_dir, "reference")
snpeff_jar = "/opt/snpeff/snpEff/snpEff.jar"

"/opt/snpeff/snpEff/snpEff.jar"

In [3]:
if !isdir(reference_dir)

    ProcessSequence.print_and_run_cmd(`unzip -o -d $input_dir $reference_dir.zip`)
    
end

In [4]:
if !isfile(snpeff_jar)
    
    throw("$snpeff is missing.")
    
end

In [5]:
ProcessSequence.check_program()

Checking program...
`which skewer`
/opt/conda/bin/skewer
`which fastqc`
/opt/conda/bin/fastqc
`which bgzip`
/opt/conda/bin/bgzip
`which tabix`
/opt/conda/bin/tabix
`which minimap2`
/opt/conda/bin/minimap2
`which samtools`
/opt/conda/bin/samtools
`which bcftools`
/opt/conda/bin/bcftools
`which kallisto`
/opt/conda/bin/kallisto
`bash -c 'source activate py2 && which configManta.py'`
/opt/conda/envs/py2/bin/configManta.py
`bash -c 'source activate py2 && which configureStrelkaGermlineWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaGermlineWorkflow.py
`bash -c 'source activate py2 && which configureStrelkaSomaticWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaSomaticWorkflow.py


In [6]:
process_dna_arguments = (
    joinpath(
        reference_dir,
        "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz",
    ),
    joinpath(reference_dir, "chromosome.bed.gz"),
    joinpath(reference_dir, "chrn_n.tsv"),
    project_json["n_job"],
    project_json["gb_memory"],
    5,
    snpeff_jar,
)

("/home/jovyan/ProcessSequence.jl/input/reference/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz", "/home/jovyan/ProcessSequence.jl/input/reference/chromosome.bed.gz", "/home/jovyan/ProcessSequence.jl/input/reference/chrn_n.tsv", 8, 40, 5, "/opt/snpeff/snpEff/snpEff.jar")

In [7]:
if all((
    in(key, keys(project_json)) for key in ("germ_dna.1.fastq.gz", "germ_dna.2.fastq.gz")
))

    ProcessSequence.process_germ_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, string("process_germ_dna_", sample_name)),
        process_dna_arguments...,
    )

end

Trimmed files already exist.
Skipping check_sequence because /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1004/check_sequence already exists.
(2020-12-06T08:31:50.057) Aligning sequence ...
pipeline(pipeline(pipeline(pipeline(`minimap2 -x sr -t 8 -K 5G -R '@RG\tID:Germ\tSM:Germ' -a /home/jovyan/ProcessSequence.jl/input/reference/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz.mmi /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1004/trim_sequence/germ-trimmed-pair1.fastq.gz /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1004/trim_sequence/germ-trimmed-pair2.fastq.gz`, stdout=`samtools sort --threads 8 -m 5G -n`), stdout=`samtools fixmate --threads 8 -m - -`), stdout=`samtools sort --threads 8 -m 5G`), stdout>Base.FileRedirect("/home/jovyan/ProcessSequence.jl/output/process_germ_dna_1004/align_sequence/germ.bam.tmp", false))


[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::7.517*1.00] loaded/built the index for 195 target sequence(s)
[M::mm_mapopt_update::7.517*1.00] mid_occ = 1000
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 195
[M::mm_idx_stat::8.580*1.00] distinct minimizers: 100167746 (38.80% are singletons); average occurrences: 5.519; average spacing: 5.607
[W::mm_bseq_read_frag2] query files have different number of records; extra records skipped.
[W::mm_bseq_read_frag2] query files have different number of records; extra records skipped.
[W::mm_bseq_read_frag2] query files have different number of records; extra records skipped.


LoadError: failed processes:
  Process(`minimap2 -x sr -t 8 -K 5G -R '@RG\tID:Germ\tSM:Germ' -a /home/jovyan/ProcessSequence.jl/input/reference/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz.mmi /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1004/trim_sequence/germ-trimmed-pair1.fastq.gz /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1004/trim_sequence/germ-trimmed-pair2.fastq.gz`, ProcessSignaled(2)) [0]
  Process(`samtools sort --threads 8 -m 5G -n`, ProcessSignaled(2)) [0]
  Process(`samtools fixmate --threads 8 -m - -`, ProcessSignaled(2)) [0]
  Process(`samtools sort --threads 8 -m 5G`, ProcessSignaled(2)) [0]


In [10]:
if all((
    in(key, keys(project_json))
    for
    key in (
        "germ_dna.1.fastq.gz",
        "germ_dna.2.fastq.gz",
        "soma_dna.1.fastq.gz",
        "soma_dna.2.fastq.gz",
    )
))

    ProcessSequence.process_soma_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, "process_soma_dna"),
        process_dna_arguments...,
    )

end

In [ ]:
if all((
    in(key, keys(project_json)) for key in ("soma_rna.1.fastq.gz", "soma_rna.2.fastq.gz")
))

    ProcessSequence.process_soma_rna(
        joinpath(project_dir, project_json["soma_rna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_rna.2.fastq.gz"]),
        joinpath(output_dir, "process_soma_rna"),
        joinpath(reference_dir, "Homo_sapiens.GRCh38.cdna.all.fa.gz"),
        project_json["n_job"],
    )

end